In [1]:
import matplotlib.pyplot as plt
import PI

figsize = (12, 4)
fig=plt.figure(figsize=figsize)


meta_params = {
    'num_of_train_dataset': 1000,
    'num_of_test_dataset': 100,
    'is_flatten': False
}

PI = PI.PIInterface(meta_params)

import torch
import torch.nn as nn
import torch.optim as optim
from MNIST_models import *

model = load_model('store/MNIST_CNN.pt')
PI.set_model(model)
# print('train acc:', PI.eval_model('train'))
# print('test acc:', PI.eval_model('test'))

1000 100
(100, 1, 28, 28)


/Users/kuotzuyang/opt/anaconda3/lib/python3.7/site-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'MNIST_models.CNN' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Users/kuotzuyang/opt/anaconda3/lib/python3.7/site-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Users/kuotzuyang/opt/anaconda3/lib/python3.7/site-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'torch.nn.modules.linear.Linear' has changed. you can retrieve the original source code by accessi

<Figure size 864x288 with 0 Axes>

In [2]:
# # visualize the benign input and corresponding adv samples 
# samples, labels = PI.train_dataset

# %matplotlib inline
# from matplotlib import pyplot as plt
# import numpy as np
# import attacker
# import time
# import warnings
# warnings.filterwarnings('ignore')


# adv_types = ['None', 'FGSM', 'JSMA', 'CWL2', 'L1PGD', 'L2PGD', 'LINFPGD', 'L2BI', 'LINFBI', 'ENL1', 'DDNL2', 'LBFGS', 'SP', 'LS', 'ST']
# print(len(adv_types))
# formatted_adv_types = [adv_type.ljust(6) for adv_type in adv_types]
# print('  '+' '.join(formatted_adv_types))

# figsize=(5, 5)
# fig=plt.figure(figsize=figsize)
# columns, num_of_rows = len(adv_types), 8

# for i in range(num_of_rows):
#     sample, label = samples[i], labels[i]
#     for col_index in range(columns):  
#         adv_type = adv_types[col_index]
#         if adv_type == 'None': 
#             plt_sample = sample 
#         else:
#             plt_sample = PI.generate_adv_img(sample, label, model, adv_type)
#             if plt_sample is None: 
#                 plt_sample = np.zeros((1, 28, 28)) 
            

#         fig.add_subplot(num_of_rows, columns, (col_index+1)+(i*columns))
#         plt.imshow(np.squeeze(plt_sample), cmap='gray')
#         plt.xticks([])
#         plt.yticks([])

# fig.savefig('out.png')

In [ ]:
import pickle

# STORE
# adv_types = ['None', 'FGSM', 'JSMA', 'CWL2', 'LINFPGD', 'LINFBI', 'ENL1', 'ST']
# adv_types = ['ENL1', 'ST']
# for adv_type in adv_types:
#     if adv_type == 'None': 
#         fn_name='store/normal.txt'
#         set_of_signatures = PI.generate_signatures() 
#     else: 
#         fn_name='store/'+adv_type+'.txt'
#         set_of_signatures = PI.generate_signatures(adv_type=adv_type)
    
#     print(adv_type, len(set_of_signatures))
#     with open(fn_name, "wb") as fp:   
#         pickle.dump(set_of_signatures, fp)
    
prefix = 'store/'    
# LOAD 
adv_types = ['None', 'FGSM', 'JSMA', 'CWL2', 'LINFPGD', 'LINFBI', 'ENL1', 'ST']
set_of_train_dataset, set_of_test_dataset = [], []

for adv_type in adv_types:
    # extract from the store file 
    if adv_type == 'None': fn_name=prefix+'normal.txt'
    else: fn_name=prefix+adv_type+'.txt'
    fp = open(fn_name, 'rb')
    set_of_signatures = pickle.load(fp)
    
    # separate and store in for later training and evaluation 
    if adv_type == 'None': split_percentage = 0.1
    else: split_percentage = 0.1
    split_line = int(len(set_of_signatures)*split_percentage)
    train_set_of_signatures, test_set_of_signatures = set_of_signatures[:split_line], set_of_signatures[split_line:]
    set_of_train_dataset.append(train_set_of_signatures)
    set_of_test_dataset.append(test_set_of_signatures)
    fp.close()
    print(adv_type, len(set_of_signatures), len(train_set_of_signatures), len(test_set_of_signatures))
    
print()
filtered_adv_types = ['None', 'LINFPGD']
filtered_set_of_train_dataset = []
for adv_type in filtered_adv_types:
    # extract from the store file 
    if adv_type == 'None': fn_name=prefix+'normal.txt'
    else: fn_name=prefix+adv_type+'.txt'
    fp = open(fn_name, 'rb')
    set_of_signatures = pickle.load(fp)
    
    # separate and store in for later training and evaluation 
    if adv_type == 'None': split_percentage = 0.8
    else: split_percentage = 0.8
    split_line = int(len(set_of_signatures)*split_percentage)
    train_set_of_signatures, test_set_of_signatures = set_of_signatures[:split_line], set_of_signatures[split_line:]
    filtered_set_of_train_dataset.append(train_set_of_signatures)
    fp.close()
    print(adv_type, len(set_of_signatures), len(train_set_of_signatures), len(test_set_of_signatures))



None 1000 100 900
FGSM 813 81 732
JSMA 966 96 870
CWL2 877 87 790
LINFPGD 978 97 881
LINFBI 970 97 873
ENL1 1000 100 900


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch

guard_model = Guard()
guard_model.train()

# set_of_train_dataset = filtered_set_of_train_dataset
epoches = 1

train_accs, test_accs, losses, set_train_sub_accs, set_test_sub_accs = train_guard_model(guard_model, set_of_train_dataset, set_of_test_dataset, adv_types, epoches)
losses = [x.item() for x in losses]
guard_model.eval()
# guard_model = torch.load('store/guard.pt')

In [ ]:
# import copy
# from scipy.ndimage.filters import gaussian_filter1d

# print('a')
# smooth_train_accs = gaussian_filter1d(train_accs, sigma=1)
# smooth_test_accs = gaussian_filter1d(test_accs, sigma=1)
# smooth_losses = gaussian_filter1d(losses, sigma=1)
# norm_smooth_losses = [float(i)/max(smooth_losses) for i in smooth_losses]
# print('b')

# fig = plt.figure()

# print('c')
# # summarize history for accuracy
# plt.plot(smooth_train_accs)
# plt.plot(smooth_test_accs)
# plt.plot(norm_smooth_losses)
# plt.title('model accuracy and loss')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train acc', 'test acc', 'loss'], loc='lower right')
# plt.ylim([0, 1.05])

# print('d')
# # plt.savefig('outputs.jpg')
# plt.show()

In [ ]:
# test_guard_model(guard_model, set_of_test_dataset, adv_types)

In [ ]:
figsize = (12, 4)
fig=plt.figure(figsize=figsize)


print('breakpoint a')
num_of_epoches = len(set_train_sub_accs)
num_of_adv_types = len(set_train_sub_accs[0])
fig.add_subplot(1, 2, 1)
for i in range(num_of_adv_types):
    adv_type_accs = [sub_accs[i] for sub_accs in set_train_sub_accs]
    smooth_adv_type_accs = gaussian_filter1d(adv_type_accs, sigma=1)
    plt.plot(smooth_adv_type_accs)
    
print('breakpoint b')
# plt.legend(filtered_adv_types, loc='lower right')
plt.legend(adv_types, loc='lower right')
plt.title('train')
plt.ylim([0, 1.05])


print('breakpoint c')
num_of_epoches = len(set_test_sub_accs)
num_of_adv_types = len(set_test_sub_accs[0])
fig.add_subplot(1, 2, 2)
for i in range(num_of_adv_types):
    adv_type_accs = [sub_accs[i] for sub_accs in set_test_sub_accs]
    smooth_adv_type_accs = gaussian_filter1d(adv_type_accs, sigma=1)
    plt.plot(smooth_adv_type_accs)
    
plt.legend(adv_types, loc='lower right')
plt.title('test')
plt.ylim([0, 1.05])

# plt.savefig('outputs.jpg')
plt.show()